## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EE8E259000>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EE8E25A470>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke(
    [
        HumanMessage(content='Hi, My name is vaibhav baneshi and I am an AI and ML engineer.'),
        AIMessage(content="Hello Vaibhav Baneshi! \n\nIt's great to meet another AI and ML engineer.  \n\nWhat kind of work do you specialize in? I'm always interested in learning about different applications of AI and ML.\n"),
        HumanMessage(content="What's my name and what do i do?")
    ]
)

AIMessage(content="You said your name is Vaibhav Baneshi and that you are an AI and ML engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 99, 'total_tokens': 152, 'completion_time': 0.096363636, 'prompt_time': 0.00426342, 'queue_time': 0.233133123, 'total_time': 0.100627056}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-47e7db79-f207-453e-a7fd-689f752e9efd-0', usage_metadata={'input_tokens': 99, 'output_tokens': 53, 'total_tokens': 152})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [12]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [13]:
config={"configurable":{"session_id":"chat1"}}

In [14]:
response=with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Vaibhav Baneshi and I am a Chief AI Engineer")
    ],
    config=config
)

response.content

"It's nice to meet you, Vaibhav Baneshi! That's a fascinating title - Chief AI Engineer. \n\nWhat kind of work are you involved in with AI? I'm always interested in learning more about how people are using this technology to make a difference.\n"

In [15]:
response=with_message_history.invoke(
    [
        HumanMessage(content="What's my name and what do i do?")
    ],
    config=config
)

response.content

"According to our conversation, your name is Vaibhav Baneshi and you are a Chief AI Engineer.  😄 \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm eager to learn more!\n"

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [22]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ] 
)

chain=prompt|model

In [26]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Vaibhav")]}) 

AIMessage(content="Hi Vaibhav! Nice to meet you. \n\nI'm happy to help with any questions you have. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 33, 'total_tokens': 67, 'completion_time': 0.061818182, 'prompt_time': 0.002349825, 'queue_time': 0.23047842, 'total_time': 0.064168007}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-95b0b773-15e8-4a24-950c-9b57dc5f310f-0', usage_metadata={'input_tokens': 33, 'output_tokens': 34, 'total_tokens': 67})

In [24]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [25]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Vaibhav")],
    config=config
)

response.content

"Hi Vaibhav, it's great to meet you!  \n\nWhat can I do for you today? 😊  \n\n"

In [27]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [28]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Vaibhav")],"language":"Hindi"})
response.content

'नमस्ते Vaibhav! मुझे बहुत खुशी है आपसे मिलने में।  \n\nआप मुझसे जो भी पूछना चाहें, मैं अपनी पूरी क्षमता से उत्तर दूंगा। 😊\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [29]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [32]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Vaibhav")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते, वैभव! 😊  \n\nआप कैसे हैं? \n'

In [33]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

response.content

'आपका नाम वैभव है। 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [53]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=75,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [54]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You said you like vanilla ice cream!  🍦  \n\n\n\nDo you want to talk about ice cream more?\n'

In [50]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2" 😊  \n\n\n\n'

In [51]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob!  🍦  😊 \n'